In [1]:
# dmpからinpファイルの作成
import os
import glob

data_dir = "data/15dCn/C_1_orient/*.dmp"
out_dir = data_dir.split("*.dmp")[0].replace("data", "result")
os.makedirs(out_dir, exist_ok=True)
paths = [p for p in glob.glob(data_dir)]

def orient_mono_a_b(input_path, output_path, a_or_b):
    # Read the input file
    with open(input_path, 'r') as file:
        lines = file.readlines()

    # Process the file with continuous atom numbering
    processed_lines = []
    atom_number = 0  # Counter for continuous atom numbering

    for line in lines:
        # Skip the specified lines at the beginning
        if line.strip().startswith('!') or line.strip().startswith('Units'):
            continue
        
        # Replace atom with atom + atom_number and "Rank  4" with "Limit  4  Type X"
        if "Rank  4" in line:
            atom = line.split()[0]
            atom_number += 1
            new_line = line.replace(atom, atom + str(atom_number))
            new_line = new_line.replace("Rank  4", f"Limit  4  Type {atom}")
        else:
            new_line = line
        processed_lines.append(new_line)

    # Add the specified line at the beginning and end of the file
    if a_or_b == "monoa":
        processed_lines.insert(0, "Molecule monomer1 at 0.0 0.0 0.0 rotated 0.0 0.0 0.0\n")
    elif a_or_b == "monob":
        processed_lines.insert(0, "Molecule monomer2 at x y z rotated alpha beta gamma\n")
        
    processed_lines.append("end\n")

    # Write the processed content to the output file
    with open(output_path, 'w') as file:
        file.writelines(processed_lines)
        
          
for path in paths:
    a_or_b = path.split("/")[-1].split("_")[0]
    file_name = path.split("/")[-1].split(".dmp")[0]
    input_path = path
    output_path_for_mp = out_dir + "mpole_" + file_name + ".inp"
    orient_mono_a_b(input_path, output_path_for_mp, a_or_b)

In [2]:
# dat2の作成
data_dir = out_dir + "*.inp"
out_dir = data_dir.split("*.inp")[0]

paths = [p for p in glob.glob(data_dir)]

H_flag = True

def make_dat2(atom_list, atom_type_list, output_path, H_flag):
    processed_lines = []
    O_line = [' 0  0  0  0\n', ' 0  6  0  0\n', ' 0  0  6  0\n', ' 0  0  0  6\n']
    C_line = [' 0  0  0  0\n', ' 0  10  0  0\n', ' 0  0  10  0\n', ' 0  0  0  10\n']
    N_line = [' 0  0  0  0\n', ' 0  8  0  0\n', ' 0  0  8  0\n', ' 0  0  0  8\n']
    H_line = [' 0  0  0  0\n', ' 0  3  0  0\n', ' 0  0  3  0\n', ' 0  0  0  3\n']

    for i, atom in enumerate(atom_list):
        atom_description = "Read rank 1 sites " + atom + "\n"

        if H_flag and atom_type_list[i] == "H":
            processed_lines.append(atom_description)
            processed_lines.extend(H_line)

        elif atom_type_list[i] == "C":
            processed_lines.append(atom_description)
            processed_lines.extend(C_line)

        elif atom_type_list[i] == "N":
            processed_lines.append(atom_description)
            processed_lines.extend(N_line)

        elif atom_type_list[i] == "O":
            processed_lines.append(atom_description)
            processed_lines.extend(O_line)

    with open(output_path, 'w') as file:
        file.writelines(processed_lines)

# まずはorient用に作成したinpファイルから原子を取り出す
# atom_list = []
# atom_type_list = []

for path in paths:
    file_name = path.split("/")[-1].split(".dmp")[0]
    input_path = path
    output_path = path.split(file_name)[0] + "dat2" + file_name.replace("mpole", "")
    atom_list = []
    atom_type_list = []
    with open(input_path, 'r') as file:
        lines = file.readlines()
        # Process the file with continuous atom numbering
        for line in lines:

            # Replace atom with atom + atom_number and "Rank  4" with "Limit  4  Type X"
            if "Limit  4  Type" in line:
                atom_and_num = line.split()[0]
                atom_type = line.split()[-1]
                atom_list.append(atom_and_num)
                atom_type_list.append(atom_type)
    
    make_dat2(atom_list, atom_type_list, output_path, H_flag)

    

In [53]:
# resultフォルダ内のdat2とinpからdim3.inpを作成
# __monoa_inp__などを書き換えるだけ
import os
import glob

# フォルダの指定
data_dir = out_dir + "*.inp"
dim3_out_dir = out_dir + "dim3/"
os.makedirs(dim3_out_dir, exist_ok=True)

paths = [p for p in glob.glob(data_dir)]

mp_monoa_paths = []
mp_monob_paths = []
dat2_monoa_paths = []
dat2_monob_paths = []

for path in paths:
    file_name = path.split("/")[-1]
    if 'dat2_monoa' in file_name:
        dat2_monoa_paths.append(path)
    elif 'dat2_monob' in file_name:
        dat2_monob_paths.append(path)
    elif "mpole_monoa" in file_name:
        mp_monoa_paths.append(path)
    elif 'mpole_monob' in file_name:
        mp_monob_paths.append(path)
        

mp_monoa_paths.sort()
mp_monob_paths.sort()
dat2_monoa_paths.sort()
dat2_monob_paths.sort()
        
# ファイルの雛形
initial_content = ['Allocate\n',
 '   sites 10000 polarizable 10000\n',
 'end\n',
 'Variables\n',
 'x  0.0 A\n',
 'y  0.0 A\n',
 'z  0.0 A\n',
 'alpha  0.0 D\n',
 'beta   0.0 D\n',
 'gamma  0.0 D\n',
 'end\n',
 'Types\n',
 'H Z 1  \n',
 'C Z 6 \n',
 'O Z 8 \n',
 'N Z 7 \n',
 'CL Z 17 \n',
 'end\n',
 '__monoa_inp__\n',
 '__monob_inp__\n',
 'unit bohr \n',
 'pairs\n',
 ' C  C       rho     alpha     C6\n',
 ' 00 00 0    5.98    1.947     30.55\n',
 ' end\n',
 ' C  H       rho     alpha     C6\n',
 ' 00 00 0    4.94    2.085      8.56\n',
 ' end\n',
 ' C  O       rho     alpha     C6\n',
 ' 00 00 0    5.65    2.069     24.63\n',
 ' end\n',
 ' C  N       rho     alpha     C6\n',
 ' 00 00 0    5.77    1.974     24.05\n',
 ' end\n',
 ' C  CL      rho     alpha     C6\n',
 ' 00 00 0    6.55    1.556     76.56\n',
 ' end\n',
 ' H  H       rho     alpha     C6\n',
 ' 00 00 0    3.95    2.270      2.10\n',
 ' end\n',
 ' H  O       rho     alpha     C6\n',
 ' 00 00 0    4.62    2.238      6.39\n',
 ' end\n',
 ' H  N       rho     alpha     C6\n',
 ' 00 00 0    4.74    2.117      6.60\n',
 ' end\n',
 ' H  CL      rho     alpha     C6\n',
 ' 00 00 0    5.47    1.625     23.37\n',
 ' end\n',
 ' O  O       rho     alpha     C6\n',
 ' 00 00 0    5.30    2.212     18.83\n',
 ' end\n',
 ' O  N       rho     alpha     C6\n',
 ' 00 00 0    5.44    2.101     19.08\n',
 ' end\n',
 ' O  CL      rho     alpha     C6\n',
 ' 00 00 0    6.37    1.553     63.82 \n',
 ' end\n',
 ' N  N       rho     alpha     C6\n',
 ' 00 00 0    5.55    2.0       18.8\n',
 ' end\n',
 ' N  CL      rho     alpha     C6\n',
 ' 00 00 0    6.49    1.498     63.76 \n',
 ' end\n',
 ' CL CL      rho     alpha     C6\n',
 ' 00 00 0    7.43    1.197    216.27\n',
 ' end\n',
 'end\n',
 'Polarizabilities for monomer1\n',
 '__dat2_monoa__\n',
 'end\n',
 'Polarizabilities for monomer2\n',
 '__dat2_monob__\n',
 'end\n',
 'switch induce on\n',
 'unit  kj/mol \n',
 'unit angstrom \n',
 'energy details  \n',
 'finish']

def find_file_by_pair(pair, file_list):
    """指定されたペアに該当するファイル名をリストから探す"""
    for file in file_list:
        if pair in file:
            return file
    return None

pair_len = len(mp_monoa_paths)
for i in range(pair_len):
    pair_n = "pair_" + str(i + 1)
    mp_monoa_pair_n = find_file_by_pair(pair_n, mp_monoa_paths)
    mp_monob_pair_n = find_file_by_pair(pair_n, mp_monob_paths)
    dat2_monoa_pair_n = find_file_by_pair(pair_n, dat2_monoa_paths)
    dat2_monob_pair_n = find_file_by_pair(pair_n, dat2_monob_paths)
    
    with open(mp_monoa_pair_n, 'r') as f:
        mp_monoa_pair_n_data = f.read()
    with open(mp_monob_pair_n, 'r') as f:
        mp_monob_pair_n_data = f.read()
    with open(dat2_monoa_pair_n, 'r') as f:
        dat2_monoa_pair_n_data = f.read()
    with open(dat2_monob_pair_n, 'r') as f:
        dat2_monob_pair_n_data = f.read()
    
    
    output_file_path = dim3_out_dir + 'dim3_' + pair_n + '.inp'
    modified_data = []
    with open(output_file_path, 'w') as output_file:
        data = initial_content.copy()
        for line in data:
            modified_line = line.replace("__monoa_inp__\n", mp_monoa_pair_n_data).replace("__monob_inp__\n", mp_monob_pair_n_data).replace("__dat2_monoa__\n", dat2_monoa_pair_n_data).replace("__dat2_monob__\n", dat2_monob_pair_n_data)
            modified_data.append(modified_line)
        
        for line in modified_data:
            output_file.write(line)

In [54]:
# shファイル作成
# orient-5.0.09-ng < dim3.inp > dim3.out &

search_path = dim3_out_dir + "*.inp"
paths = [p for p in glob.glob(search_path)]

output_file = dim3_out_dir + "orient.sh"

with open(output_file,"w") as f:
    f.write("#! /bin/sh" + "\n")

for path in paths:
    file_name = path.split("/")[-1]
    with open(output_file,"a") as f:
        text = "orient-5.0.09-ng < " + file_name + " > " + file_name.replace("inp", "out") + " &" + "\n"
        f.write(text)
    
